# Generate Input- Parse Ship Met data

This script will read in the raw _.MET_ data files to generate a merged dataset for use in the rest of the model.

After this script is run, please run the ___Merge Ship Data___.

In [ ]:
## Install the package if needed:
#install.packages('Mcomp', repos='http://cran.us.r-project.org')

## Load the relevent packages
library(ncdf4)  # For reading in the NCEP wind fields
#library(R.matlab)  # If you need to read in matlab .mat files
library(openxlsx)  # If you need to read in .xlsx files
#library(rNOMADS)  # For reading grib2 data files (NOMADS data for instance)
#library(rGDAL)  #
library(RColorBrewer)
library(compiler)  # required for JIT (below)

## Enable compilation (speed gain?)
enableJIT(3)

## Helper function for converting the date time stamps.
conv_excel_time = function(x, tz='US/Pacific') {
    as.POSIXct(as.Date(x,origin="1899-12-30", tz=tz))
}

get.qual.pal = function(n=100, pal='Accent') {
    colorRampPalette(brewer.pal(8, pal))(n)
}

get.seq.pal = function(n=100, pal='YlOrRd') {
    colorRampPalette(rev(brewer.pal(11, pal)))(n)
}

get.div.pal = function(n=100, pal='Spectral') {
    colorRampPalette(rev(brewer.pal(11, pal)))(n)
}

make.div.pal = function(x=100, n, pal='Spectral') {
    get.div.pal(n, pal=pal)[as.numeric(cut(x, breaks = n))]
}

## Setup file information

In [44]:
input.dir = '../Met-data - raw/'
#input.dir = 'Raw Data/MET/'

dt.format = "%m/%d/%y %I:%M %p"

## Generate list of files in the directory
file.list = list.files(input.dir)
file.list = file.list[grepl('.MET', file.list)]

## Print total number of files in each category
print(paste('Number of files in file.list:', length(file.list)))

[1] "Number of files in file.list: 33"


In [48]:
processFile = function(filepath) {
    con = file(filepath, "r")
    i = 0
    
    while ( TRUE ) {
        i = i + 1
        line = readLines(con, n = 1)
        if ( length(line) == 0 ) {
          break
        }
        
        ## Get the header
        if (i == 4) {
            line = substring(line, 2)
            header = strsplit(line, split = ' ')[[1]]
            header = header[which(sapply(header, function(x) {nchar(x) > 0}))]
        }
        if (i > 4) {
            if (nchar(line) > 5) {
                s = strsplit(line, split = ' ')[[1]]
                s = s[which(sapply(s, function(x) {nchar(x) > 0}))]
                if(exists('dat')) {
                    dat = rbind(dat, as.numeric(s))
                } else {
                    dat = data.frame(t(as.numeric(s)))
                }
            }
        }
    }
    colnames(dat) = header
    close(con)
    return(dat)
}

## Read in the data

In [49]:
data = processFile(paste0(input.dir, file.list[1]))

In [50]:
str(data)

'data.frame':	1002 obs. of  90 variables:
 $ Time: num  194749 194849 194904 194919 195004 ...
 $ AT  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ BP  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ BS  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ PR  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ RH  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ RT  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ DP  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ LD  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ LB  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ LT  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ LW  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ SW  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ PA  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ WS  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ WD  : num  -99 -99 -99 -99 -99 -99 -99 -99 -99 -99 ...
 $ TW  : num  -99 -99 -99 -99 -99 -

## Time the data columns